In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.Session(config=config))

/home/janek/.local/share/virtualenvs/rosetta-WKmHhL03/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import keras
import keras.layers as L
from keras.models import Model
import numpy as np
import pandas as pd

np.random.seed(0)

In [3]:
START = '^'
END = '$'

SIZE = 100_000
LATENT_DIM = 128
EMBEDDING_DIM = 32

In [4]:
def create_equations_df(size, min_value=0, max_value=9999, operations={'+': np.add, '-': np.subtract}):
    df = pd.DataFrame()
    df['a'] = np.random.randint(low=min_value, high=max_value, size=size)
    df['b'] = np.random.randint(low=min_value, high=max_value, size=size)
    df['op'] = np.random.choice(list(operations.keys()), size)
    df['result'] = np.zeros(size, dtype='int')
    for symbol, calc in operations.items():
        df.loc[df.op == symbol, 'result'] = calc(df[df.op == symbol]['a'], df[df.op == symbol]['b'])
        
    df['input_texts'] = START + df.a.astype(str) + df.op + df.b.astype(str) + END
    df['target_texts'] = START + df.result.astype(str) + END
    return df

In [5]:
df = create_equations_df(SIZE)

In [6]:
corpus = pd.concat([df.input_texts, df.target_texts])

In [7]:
tokenizer = keras.preprocessing.text.Tokenizer(num_words=None, filters=None, char_level=True)
tokenizer.fit_on_texts(corpus)
df['input_sequences'] = tokenizer.texts_to_sequences(df.input_texts)
df['target_sequences'] = tokenizer.texts_to_sequences(df.target_texts)

In [8]:
X = keras.preprocessing.sequence.pad_sequences(df.input_sequences, padding='post').astype(np.float32)
# y = keras.preprocessing.sequence.pad_sequences(df.target_sequences)
y = df.result.astype(np.float32)

In [9]:
encoder_inputs = L.Input(shape=(X.shape[1], ), dtype='float32')
#embedding = L.Embedding(len(tokenizer.word_index), EMBEDDING_DIM)(encoder_inputs)
dense = L.Dense(1, activation='relu')(encoder_inputs) #(embedding)

model = Model(inputs=encoder_inputs, outputs=dense)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 11)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 12        
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [11]:
model.fit(X, y, validation_split=0.2, epochs=50)

Train on 80000 samples, validate on 20000 samples
Epoch 1/50
80000/80000 [==============================] - 4s 49us/step - loss: 65778341.6776 - val_loss: 65221600.7680
Epoch 2/50
80000/80000 [==============================] - 3s 41us/step - loss: 63981617.9400 - val_loss: 63472755.2288
Epoch 3/50
80000/80000 [==============================] - 3s 40us/step - loss: 62258186.5376 - val_loss: 61794575.7408
Epoch 4/50
80000/80000 [==============================] - 3s 42us/step - loss: 60603751.2272 - val_loss: 60185226.5024
Epoch 5/50
80000/80000 [==============================] - 3s 42us/step - loss: 59022104.9264 - val_loss: 58649351.2800
Epoch 6/50
80000/80000 [==============================] - 3s 40us/step - loss: 57511993.2568 - val_loss: 57183128.3488
Epoch 7/50
80000/80000 [==============================] - 3s 41us/step - loss: 56068211.1264 - val_loss: 55782570.3616
Epoch 8/50
80000/80000 [==============================] - 3s 41us/step - loss: 54690706.8000 - val_loss: 54446054.185

In [12]:
model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(['^1+2$']), padding='post', maxlen=X.shape[1]))

array([[1629.992]], dtype=float32)